In [2]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder

### validate
df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
print (f"df_val:{df_val.shape[0]}")
texts =np.concatenate ( [df_val["less_toxic"].values,df_val["more_toxic"].values] )

df_text = pd.DataFrame ({
    "text":list(set(texts))
})

print (f"df_text:{df_text.shape[0]}")
df_text.to_csv("../processed/validation_texts.csv", index=False)

df_text.head()

df_val:30108
df_text:14251


,text
0,Whatever gave you the impression that I have t...
1,Jesus god you're fucking retarded \n\nwhat a ...
2,"January 4, 2007 \nIf you continue to damage a..."
3,"""\n\nDC101 IS HIS FUCKING EMPLOYER!!! IF THAT ..."
4,"Don't see any attack, unlike the one YOU made ..."


In [3]:
text_encoder = LabelEncoder ()
text_encoder.fit(texts)

df_val["less_toxic"]=text_encoder.transform(df_val["less_toxic"])
df_val["more_toxic"]=text_encoder.transform(df_val["more_toxic"])
df_val["text_a"] = df_val.apply(lambda row: row["less_toxic"] if row["less_toxic"]  <  row["more_toxic"] else  row["more_toxic"],axis=1).astype(int)
df_val["text_b"] = df_val.apply(lambda row: row["more_toxic"] if row["less_toxic"]  <  row["more_toxic"] else  row["less_toxic"],axis=1).astype(int)
df_val["win_a"] = df_val.apply(lambda row: 0 if row["less_toxic"]  <  row["more_toxic"] else 1,axis=1)
df_val["win_b"] = df_val.apply(lambda row: 1 if row["less_toxic"]  <  row["more_toxic"] else 0,axis=1)
df_val_agg = df_val.groupby(["text_a","text_b"]).agg({"win_a":"sum","win_b":"sum"}).reset_index()

df_val_agg["less_toxic"] = df_val_agg.apply (lambda row: row["text_a"] if row["win_a"]< row["win_b"] else row["text_b"], axis=1 )
df_val_agg["more_toxic"] = df_val_agg.apply (lambda row: row["text_b"] if row["win_a"]< row["win_b"] else row["text_a"], axis=1 )
df_val_agg["count"] = df_val_agg["win_a"]+df_val_agg["win_b"]
df_val_agg["count"] = df_val_agg["win_a"]+df_val_agg["win_b"]
df_val_agg["perc_win_a"] = df_val_agg["win_a"]/df_val_agg["count"]
df_val_agg["perc_win_b"] = df_val_agg["win_b"]/df_val_agg["count"]
df_val_agg["avg_agreement"] = df_val_agg[["perc_win_a","perc_win_b"]].max(axis=1)
df_val_agg["less_toxic"] = text_encoder.inverse_transform ( df_val_agg["less_toxic"] )
df_val_agg["more_toxic"] = text_encoder.inverse_transform ( df_val_agg["more_toxic"] )
df_val_agg = df_val_agg [["less_toxic", "more_toxic", "count", "avg_agreement"]]


df_val_agg.to_csv("../processed/validation.csv", index=False)

df_val_agg.head()

,less_toxic,more_toxic,count,avg_agreement
0,"""::you wouldn't know Makoto from Street Fighte...",\n\nThe comment directly above this one are fr...,3,0.666667
1,I just read what you wrote; neither of the tw...,\n\nwhy should people have to read crap posted...,3,1.000000
2,"""\n\nI don't know which """"personal attacks"""" y...",\n\nwhy should people have to read crap posted...,3,1.000000
3,"Yes it is. This is NOT an encyclopedia, it is ...",\n\nwhy should people have to read crap posted...,3,0.666667
4,people like you are the reason wikipedia is us...,\nGo F yourself you cottonheadednittymuggins.,3,0.666667
